2019 Census Data for Selected Variables - Baltimore City

In [1]:
#pip install us

In [2]:
# pip install censusgeocode
# pip install censusdata

In [3]:
# pip install git+git://github.com/geopandas/geopandas.git

In [4]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [5]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
# import censusdata 
import censusgeocode as cg
#import gmaps

# Census & gmaps API Keys
from config import api_key
c = Census(api_key, year=2019)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [6]:
# testing search function
#sample = censusdata.search('acs1', 2015, 'concept', 'employment')
#print(sample)


In [7]:
#censusdata.printtable(censusdata.censustable('acs1', 2019, 'B08604'))

In [8]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ex. "B23025_005E" is unemployment count
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",                                                            
                          "B15003_017E",
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_008E",
                          "B03001_003E",               
                                       ), 
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*") 

# convert to dataframe
census_pd = pd.DataFrame(census_data)
census_pd.head()
                         

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,B02001_002E,B02001_003E,B02001_008E,B03001_003E,state,county,tract
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,23.0,1741.0,0.0,0.0,24,510,260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,1683.0,169.0,99.0,88.0,24,510,230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,440.0,4254.0,0.0,94.0,24,510,260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,281.0,626.0,29.0,66.0,24,510,260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,1748.0,310.0,10.0,970.0,24,510,260800


In [9]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

Number of rows, columns:  (200, 17)


In [10]:
# Column Reordering
census_pd = census_pd.rename(columns={"B19013_001E": "Median household income",
                                      "B01003_001E": "Population",
                                      "B01002_001E": "Median age",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_008E": "Pop. 2 or more races",
                                      "B03001_003E": "Pop. Hispanic origin",               
                                      "NAME": "Name", "state": "State"})

# Add in poverty rate (Poverty Count / Population)
census_pd["Poverty rate"] = 100 * \
    census_pd["Poverty count"].astype(
        int) / census_pd["Population"].astype(int)


# Add in unemployment rate (Employment Count / Population)
census_pd["Unemployment rate"] = 100 * \
    census_pd["Unemployment count"].astype(
        int) / census_pd["Population"].astype(int)
census_pd.head()

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median household income",
                       "Per capita income", "Poverty count", "Poverty rate", "Unemployment rate", 
                       "# employed, age 16+", "Unemployment count",
                      "# persons age 25+ graduated high school", "# persons age 25+ with Bachelor's degree",
                      "Median age","Pop. white", "Pop. Black", "Pop. 2 or more races", "Pop. Hispanic origin" ]]

census_pd.head()

,State,Name,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin
0,24,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,31625.0,21744.0,436.0,24.716553,1.927438,885.0,34.0,450.0,91.0,27.9,23.0,1741.0,0.0,0.0
1,24,"Census Tract 2302, Baltimore city, Maryland",1982.0,107391.0,63973.0,151.0,7.618567,0.857719,1500.0,17.0,72.0,593.0,30.7,1683.0,169.0,99.0,88.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,62586.0,25903.0,469.0,9.821990,3.162304,2366.0,151.0,1231.0,321.0,41.3,440.0,4254.0,0.0,94.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",969.0,17000.0,19562.0,377.0,38.906089,2.063983,286.0,20.0,229.0,47.0,37.0,281.0,626.0,29.0,66.0
4,24,"Census Tract 2608, Baltimore city, Maryland",2332.0,40795.0,26895.0,481.0,20.626072,1.629503,1233.0,38.0,394.0,309.0,36.1,1748.0,310.0,10.0,970.0


In [13]:
census_pd["Poverty rate"] = census_pd["Poverty rate"].astype(float).round(1)
census_pd.head()

,State,Name,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin
0,24,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,31625.0,21744.0,436.0,24.7,1.927438,885.0,34.0,450.0,91.0,27.9,23.0,1741.0,0.0,0.0
1,24,"Census Tract 2302, Baltimore city, Maryland",1982.0,107391.0,63973.0,151.0,7.6,0.857719,1500.0,17.0,72.0,593.0,30.7,1683.0,169.0,99.0,88.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,62586.0,25903.0,469.0,9.8,3.162304,2366.0,151.0,1231.0,321.0,41.3,440.0,4254.0,0.0,94.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",969.0,17000.0,19562.0,377.0,38.9,2.063983,286.0,20.0,229.0,47.0,37.0,281.0,626.0,29.0,66.0
4,24,"Census Tract 2608, Baltimore city, Maryland",2332.0,40795.0,26895.0,481.0,20.6,1.629503,1233.0,38.0,394.0,309.0,36.1,1748.0,310.0,10.0,970.0


In [14]:
census_pd["Unemployment rate"] = census_pd["Unemployment rate"].astype(float).round(1)
census_pd.head()

,State,Name,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin
0,24,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,34.0,450.0,91.0,27.9,23.0,1741.0,0.0,0.0
1,24,"Census Tract 2302, Baltimore city, Maryland",1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,17.0,72.0,593.0,30.7,1683.0,169.0,99.0,88.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,151.0,1231.0,321.0,41.3,440.0,4254.0,0.0,94.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,20.0,229.0,47.0,37.0,281.0,626.0,29.0,66.0
4,24,"Census Tract 2608, Baltimore city, Maryland",2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,38.0,394.0,309.0,36.1,1748.0,310.0,10.0,970.0


In [15]:
census_pd.count()

State                                       200
Name                                        200
Population                                  200
Median household income                     200
Per capita income                           200
Poverty count                               200
Poverty rate                                199
Unemployment rate                           199
# employed, age 16+                         200
Unemployment count                          200
# persons age 25+ graduated high school     200
# persons age 25+ with Bachelor's degree    200
Median age                                  200
Pop. white                                  200
Pop. Black                                  200
Pop. 2 or more races                        200
Pop. Hispanic origin                        200
dtype: int64

In [16]:

census_pd[['Census_Tract', "County", "State Name"]]= census_pd['Name'].str.split(",", n=3, expand=True)
census_pd.head()

,State,Name,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Census_Tract,County,State Name
0,24,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,34.0,450.0,91.0,27.9,23.0,1741.0,0.0,0.0,Census Tract 2604.03,Baltimore city,Maryland
1,24,"Census Tract 2302, Baltimore city, Maryland",1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,17.0,72.0,593.0,30.7,1683.0,169.0,99.0,88.0,Census Tract 2302,Baltimore city,Maryland
2,24,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,151.0,1231.0,321.0,41.3,440.0,4254.0,0.0,94.0,Census Tract 2601.02,Baltimore city,Maryland
3,24,"Census Tract 2603.03, Baltimore city, Maryland",969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,20.0,229.0,47.0,37.0,281.0,626.0,29.0,66.0,Census Tract 2603.03,Baltimore city,Maryland
4,24,"Census Tract 2608, Baltimore city, Maryland",2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,38.0,394.0,309.0,36.1,1748.0,310.0,10.0,970.0,Census Tract 2608,Baltimore city,Maryland


In [17]:
census_pd = census_pd.drop(["State"], axis=1)
census_pd.head()

,Name,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Census_Tract,County,State Name
0,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,34.0,450.0,91.0,27.9,23.0,1741.0,0.0,0.0,Census Tract 2604.03,Baltimore city,Maryland
1,"Census Tract 2302, Baltimore city, Maryland",1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,17.0,72.0,593.0,30.7,1683.0,169.0,99.0,88.0,Census Tract 2302,Baltimore city,Maryland
2,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,151.0,1231.0,321.0,41.3,440.0,4254.0,0.0,94.0,Census Tract 2601.02,Baltimore city,Maryland
3,"Census Tract 2603.03, Baltimore city, Maryland",969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,20.0,229.0,47.0,37.0,281.0,626.0,29.0,66.0,Census Tract 2603.03,Baltimore city,Maryland
4,"Census Tract 2608, Baltimore city, Maryland",2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,38.0,394.0,309.0,36.1,1748.0,310.0,10.0,970.0,Census Tract 2608,Baltimore city,Maryland


In [18]:
census_pd.columns

Index(['Name', 'Population', 'Median household income', 'Per capita income',
       'Poverty count', 'Poverty rate', 'Unemployment rate',
       '# employed, age 16+', 'Unemployment count',
       '# persons age 25+ graduated high school',
       '# persons age 25+ with Bachelor's degree', 'Median age', 'Pop. white',
       'Pop. Black', 'Pop. 2 or more races', 'Pop. Hispanic origin',
       'Census_Tract', 'County', 'State Name'],
      dtype='object')

In [ ]:
# calculate daytime population; from: https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# Two methods for calculating commuter-adjusted population estimates:

#Method 1: Total resident population + (total workers working in area - workers who lived and worked in same area) - (total workers living in area - workers who lived and worked in same area) OR;
#Method 2: Total resident population + Total workers working in area - Total workers living in area.




In [19]:
census_pd["Census_Tract"] = census_pd['Census_Tract'].str.replace('Census Tract', "") 
census_pd

,Name,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Census_Tract,County,State Name
0,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,34.0,450.0,91.0,27.9,23.0,1741.0,0.0,0.0,2604.03,Baltimore city,Maryland
1,"Census Tract 2302, Baltimore city, Maryland",1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,17.0,72.0,593.0,30.7,1683.0,169.0,99.0,88.0,2302,Baltimore city,Maryland
2,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,151.0,1231.0,321.0,41.3,440.0,4254.0,0.0,94.0,2601.02,Baltimore city,Maryland
3,"Census Tract 2603.03, Baltimore city, Maryland",969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,20.0,229.0,47.0,37.0,281.0,626.0,29.0,66.0,2603.03,Baltimore city,Maryland
4,"Census Tract 2608, Baltimore city, Maryland",2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,38.0,394.0,309.0,36.1,1748.0,310.0,10.0,970.0,2608,Baltimore city,Maryland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,"Census Tract 2201, Baltimore city, Maryland",4447.0,80129.0,67973.0,539.0,12.1,1.9,2762.0,83.0,255.0,1336.0,33.1,3384.0,471.0,114.0,187.0,2201,Baltimore city,Maryland
196,"Census Tract 2303, Baltimore city, Maryland",1408.0,104250.0,61852.0,117.0,8.3,1.1,1161.0,15.0,91.0,579.0,30.7,1213.0,101.0,18.0,54.0,2303,Baltimore city,Maryland
197,"Census Tract 2502.07, Baltimore city, Maryland",2337.0,38194.0,20012.0,691.0,29.6,4.5,916.0,106.0,512.0,142.0,32.5,158.0,2000.0,123.0,156.0,2502.07,Baltimore city,Maryland
198,"Census Tract 2503.03, Baltimore city, Maryland",2229.0,38519.0,24608.0,568.0,25.5,6.7,935.0,149.0,425.0,137.0,37.0,1189.0,650.0,59.0,116.0,2503.03,Baltimore city,Maryland


In [20]:
census_pd = census_pd.drop(["State Name"], axis=1)
census_pd.head()

,Name,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Census_Tract,County
0,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,34.0,450.0,91.0,27.9,23.0,1741.0,0.0,0.0,2604.03,Baltimore city
1,"Census Tract 2302, Baltimore city, Maryland",1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,17.0,72.0,593.0,30.7,1683.0,169.0,99.0,88.0,2302,Baltimore city
2,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,151.0,1231.0,321.0,41.3,440.0,4254.0,0.0,94.0,2601.02,Baltimore city
3,"Census Tract 2603.03, Baltimore city, Maryland",969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,20.0,229.0,47.0,37.0,281.0,626.0,29.0,66.0,2603.03,Baltimore city
4,"Census Tract 2608, Baltimore city, Maryland",2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,38.0,394.0,309.0,36.1,1748.0,310.0,10.0,970.0,2608,Baltimore city


In [ ]:
# Calculations can also be performed on Series and added into DataFrames as new columns
#thousands_of_dollars = data_file_df["Amount"]/1000
#data_file_df["Thousands of Dollars"] = thousands_of_dollars

#data_file_df.head()

In [ ]:
# Converting the membership days into weeks and then adding a column to the DataFrame
#weeks = training_df["Membership (Days)"]/7
#training_df["Membership (Weeks)"] = weeks

#training_df.head()

In [ ]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_pd["Census_Tract"].unique())
tract_count

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
#census_pd.to_csv("Comm_Corridor_Stats_Draft2.csv", index = False, header=True)

In [ ]:
# Join the attributes of the dataframes together
# Source: https://geopandas.org/docs/user_guide/mergingdata.html
#va_merge = va_tract.merge(va_df, on = "GEOID")

# Show result
#print(va_merge.head(2))
3print('Shape: ', va_merge.shape)